# Practica Nba:

# Victor Marinas 

# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

**Persist:** podemos almacenar los datos en la memoria cache.
(nos permite seleccionar el nivel de almacenamiento mediante un parámetro (persist(StorageLevel.NIVEL)).)
    
**Cache:** permite guardar los datos en el cache
(Es efectivo con pequeños conjuntos de datos)

**Collect:** sirve para sacar los resultados.
(No es eficiente en grandes conjuntos de datos)

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

1.- Cargaremos las librerias, que vayamos a utilizar en la práctica.

2.-Cargaremos la base de datos, con su ruta correspondiente y sus contextos.

3.- Introducimos nuestros contexto con sc()

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

1.- Valores duplicados: utilizaremos la funcion .distinct para seleccionar valores únicos.

2.-Columnas sin información: debemos elminar o reemplazar estos datos, depende de la cantidad de datos..

3.- Pasar datos string a numericos.

4.- Escalar variables, ya que podriamos tener diferentes variables en diferentes unidades de medida.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
#Cargamos las librerias

from pyspark import SparkContext
sc = SparkContext()

In [13]:
sqlCont = SQLContext(sc)

In [2]:
datos = './partidosLigaNBA.csv'
data_2 = sc.textFile(datos)

In [3]:
#Observamos los datos cargados

data_2.take(5)


['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93',
 'Tue, Oct 30, 2007:"7:00 pm":Portland Trail Blazers:97:San Antonio Spurs:106',
 'Wed, Oct 31, 2007:"8:00 pm":Dallas Mavericks:92:Cleveland Cavaliers:74']

In [4]:
#Limpiamos los datos: Realizamos el parseado, ya que los datos van separados por :

clean_data = data_2.map(lambda x:x.split(':'))
clean_data.take(5)

[['Date', 'Start..ET.', 'Visitor.Neutral', 'PTS', 'Home.Neutral', 'PTS.1'],
 ['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96'],
 ['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Houston Rockets',
  '95',
  'Los Angeles Lakers',
  '93'],
 ['Tue, Oct 30, 2007',
  '"7',
  '00 pm"',
  'Portland Trail Blazers',
  '97',
  'San Antonio Spurs',
  '106'],
 ['Wed, Oct 31, 2007',
  '"8',
  '00 pm"',
  'Dallas Mavericks',
  '92',
  'Cleveland Cavaliers',
  '74']]

In [5]:
#ELIMINO LOS HEADER
header = clean_data.first()

In [6]:
clean_data = data_2.map(lambda x:x.split(':')).filter(lambda x:'Playoffs' not in x[0]).filter(lambda x: x != header)
clean_data.take(5)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96'],
 ['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Houston Rockets',
  '95',
  'Los Angeles Lakers',
  '93'],
 ['Tue, Oct 30, 2007',
  '"7',
  '00 pm"',
  'Portland Trail Blazers',
  '97',
  'San Antonio Spurs',
  '106'],
 ['Wed, Oct 31, 2007',
  '"8',
  '00 pm"',
  'Dallas Mavericks',
  '92',
  'Cleveland Cavaliers',
  '74'],
 ['Wed, Oct 31, 2007',
  '"8',
  '30 pm"',
  'Seattle SuperSonics',
  '103',
  'Denver Nuggets',
  '120']]

## Quinta tarea: Media de la diferencia de puntos por año

In [7]:
#Cargamos las librerias correspondientes


from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt

In [20]:
#Creamos un dataframe, con los equipos (local y visitante), con la fecha y hora y la puntucion:


data_frame = clean_data.map(lambda x:Row(fecha = dt.datetime.strptime((x[0]),'%a, %b %d, %Y').strftime('%m/%d/%Y'),
                                         hora_partido = (x[1] +' : '+ x[2]),
                                         local_team = x[5],
                                         visitor_team = x[3],
                                         visitor_score =int(x[4]),
                                         local_score =int(x[6])))




In [21]:
new_dataframe = sqlCont.createDataFrame(data_frame)
data_frame.registerTempTable('interactions')

AttributeError: 'PipelinedRDD' object has no attribute 'registerTempTable'

In [22]:
#Mostramos el nuevo Dataframe

new_dataframe.show(7)

+----------+------------+-----------+--------------------+-------------+--------------------+
|     fecha|hora_partido|local_score|          local_team|visitor_score|        visitor_team|
+----------+------------+-----------+--------------------+-------------+--------------------+
|10/30/2007| "7 : 30 pm"|         96|Golden State Warr...|          117|           Utah Jazz|
|10/30/2007| "7 : 30 pm"|         93|  Los Angeles Lakers|           95|     Houston Rockets|
|10/30/2007| "7 : 00 pm"|        106|   San Antonio Spurs|           97|Portland Trail Bl...|
|10/31/2007| "8 : 00 pm"|         74| Cleveland Cavaliers|           92|    Dallas Mavericks|
|10/31/2007| "8 : 30 pm"|        120|      Denver Nuggets|          103| Seattle SuperSonics|
|10/31/2007| "7 : 00 pm"|        119|      Indiana Pacers|          110|  Washington Wizards|
|10/31/2007| "7 : 00 pm"|        101|   Memphis Grizzlies|          104|   San Antonio Spurs|
+----------+------------+-----------+--------------------+--

In [23]:
#En el nuevo dataframe, creamos la columna "diferencia", que es la resta del local score y visitor score
#mostramos las 10 primeras filas

new_dataframe = new_dataframe.withColumn('diferencia', abs(new_dataframe.local_score - new_dataframe.visitor_score)).show(11)

+----------+------------+-----------+--------------------+-------------+--------------------+----------+
|     fecha|hora_partido|local_score|          local_team|visitor_score|        visitor_team|diferencia|
+----------+------------+-----------+--------------------+-------------+--------------------+----------+
|10/30/2007| "7 : 30 pm"|         96|Golden State Warr...|          117|           Utah Jazz|        21|
|10/30/2007| "7 : 30 pm"|         93|  Los Angeles Lakers|           95|     Houston Rockets|         2|
|10/30/2007| "7 : 00 pm"|        106|   San Antonio Spurs|           97|Portland Trail Bl...|         9|
|10/31/2007| "8 : 00 pm"|         74| Cleveland Cavaliers|           92|    Dallas Mavericks|        18|
|10/31/2007| "8 : 30 pm"|        120|      Denver Nuggets|          103| Seattle SuperSonics|        17|
|10/31/2007| "7 : 00 pm"|        119|      Indiana Pacers|          110|  Washington Wizards|         9|
|10/31/2007| "7 : 00 pm"|        101|   Memphis Grizzli

In [24]:
# Quremos calcular la diferencia de puntos por año, por RDD:

In [25]:
datos_diferencia = clean_data.map(lambda x: (x[0][len(x)-11:18], __builtin__.diferencia(int(x[4]) - int(x[6]))))

tupla_vacia = (0, 0)
datos_diferencia.aggregateByKey(tupla_vacia, lambda a,b: (a[0] + b,    a[1] + 1),
                                 lambda a,b: (a[0] + b[0], a[1] + b[1]))\
                                .mapValues(lambda x: x[0]/x[1])\
                                .sortBy(lambda x: x[0], False)\
                                .collect()


#muestra la diferencia de puntos, de cada año desde 2007 a 2017

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 10.0 failed 1 times, most recent failure: Lost task 1.0 in stage 10.0 (TID 11, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspark/

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [ ]:
#Esto se debe a que los equipos que se clasifican para Play-off juegan más partidos.
# No juegan los mismos partidos en casa que de visitante, ya que en el playoff,
#hay eliminatorias que pueden jugar más partidos en casa que de visitante o al reves.




In [26]:
from operator import add

In [28]:
#Contamos el numero de veces que aparece un equipo tanto de local como de visitante

visitor_team = clean_data.map(lambda y: (y[3], 1))
local_team = clean_data.map(lambda y: (y[5], 1))

Local = local_team.reduceByKey(add).sortBy(lambda x: x[1], False).collect()
Visitantes = visitor_team.reduceByKey(add).sortBy(lambda x: x[1], False).collect()

In [29]:
#Numero de veces que jugo el equipo de local.


Local

[('San Antonio Spurs', 467),
 ('Boston Celtics', 467),
 ('Miami Heat', 461),
 ('Los Angeles Lakers', 450),
 ('Cleveland Cavaliers', 449),
 ('Atlanta Hawks', 448),
 ('Golden State Warriors', 445),
 ('Chicago Bulls', 436),
 ('Indiana Pacers', 434),
 ('Houston Rockets', 434),
 ('Memphis Grizzlies', 433),
 ('Orlando Magic', 431),
 ('Dallas Mavericks', 431),
 ('Los Angeles Clippers', 431),
 ('Toronto Raptors', 426),
 ('Portland Trail Blazers', 425),
 ('Denver Nuggets', 424),
 ('Utah Jazz', 422),
 ('Washington Wizards', 421),
 ('Philadelphia 76ers', 416),
 ('Detroit Pistons', 415),
 ('Milwaukee Bucks', 413),
 ('Phoenix Suns', 412),
 ('New York Knicks', 412),
 ('Oklahoma City Thunder', 410),
 ('Minnesota Timberwolves', 402),
 ('Sacramento Kings', 402),
 ('Charlotte Bobcats', 283),
 ('New Orleans Hornets', 250),
 ('Brooklyn Nets', 217),
 ('New Jersey Nets', 197),
 ('New Orleans Pelicans', 166),
 ('Charlotte Hornets', 126),
 ('Seattle SuperSonics', 41)]

In [30]:
#Numero de veces que jugo el equipo de visitante

Visitantes

[('San Antonio Spurs', 466),
 ('Boston Celtics', 463),
 ('Miami Heat', 456),
 ('Cleveland Cavaliers', 452),
 ('Atlanta Hawks', 448),
 ('Los Angeles Lakers', 447),
 ('Golden State Warriors', 440),
 ('Chicago Bulls', 437),
 ('Dallas Mavericks', 436),
 ('Memphis Grizzlies', 434),
 ('Houston Rockets', 434),
 ('Indiana Pacers', 434),
 ('Orlando Magic', 432),
 ('Los Angeles Clippers', 430),
 ('Portland Trail Blazers', 428),
 ('Utah Jazz', 424),
 ('Toronto Raptors', 424),
 ('Denver Nuggets', 424),
 ('Washington Wizards', 423),
 ('Philadelphia 76ers', 418),
 ('Detroit Pistons', 414),
 ('Milwaukee Bucks', 414),
 ('Phoenix Suns', 413),
 ('New York Knicks', 413),
 ('Oklahoma City Thunder', 408),
 ('Minnesota Timberwolves', 402),
 ('Sacramento Kings', 402),
 ('Charlotte Bobcats', 283),
 ('New Orleans Hornets', 249),
 ('Brooklyn Nets', 218),
 ('New Jersey Nets', 197),
 ('New Orleans Pelicans', 166),
 ('Charlotte Hornets', 127),
 ('Seattle SuperSonics', 41)]

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

In [31]:
clevelant_local = clean_data.filter(lambda x:x[5] == 'Cleveland Cavaliers').filter(lambda x:'Jan' in x[0]).filter(lambda x: int(x[4])<int(x[6]))
clevelant_local.count()

42

In [35]:
clevelant_visitante = clean_data.filter(lambda x:x[3] == 'Cleveland Cavaliers').filter(lambda x:'Jan' in x[0]).filter(lambda x: int(x[4])>int(x[6]))

clevelant_visitante.count()

41

In [36]:
#Sumamos las victorias de visitante y de local

clevelant_local.count() + clevelant_visitante.count()

#El resultado es 83

83

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

In [38]:
#Golden State Warriors Local


warriors_local = clean_data.filter(lambda x:x[3] == 'Golden State Warriors').filter(lambda x: int(x[4])>int(x[6])).map(lambda x: (x[3],1)).reduceByKey(add).sortByKey().take(1)

warriors_local

[('Golden State Warriors', 215)]

In [39]:
#Golden State Warriors Visitante

warriors_visitante = clean_data.filter(lambda x:x[5] == 'Golden State Warriors').filter(lambda x:int(x[6])>int(x[4])).map(lambda x : (x[5],1)).reduceByKey(add).sortByKey().take(1)

warriors_visitante

[('Golden State Warriors', 308)]

In [40]:
warriors_local + warriors_visitante



[('Golden State Warriors', 215), ('Golden State Warriors', 308)]

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

In [ ]:
# no se hacerlo......

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

In [ ]:
# no se hacerlo......

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com